In [33]:

#parameters
directory ="/Users/weichen/Downloads/"
file_name = "title_content_2gram_processed.csv"
to_file_name = directory+ "vector_title_content.csv"#####
text_column_names = ["title_new","content_new"]
label_column_name = "label"
one_hot_column_name = "publication"

import pandas as pd
import numpy as np
def drop_nan_epmtystr(df,subset=['content_new']):
    df.replace(r'^\s*$', np.nan, regex=True,inplace=True)
    df.replace('', np.nan, inplace=True)
    print("datasize: ",len(df))
    df.dropna(subset=subset, inplace=True)
    print("Drop empty content...")
    print("new datasize: ",len(df))
    return df

#read data
df = pd.read_csv(directory+file_name)
#always drop so code doesnt crash later
df = drop_nan_epmtystr(df,text_column_names)
df.head()

max_feature = [441,32]
# min_df = [.1,.001]
vectorizers = []
for index, column_name in enumerate(text_column_names):
    df2 = df.copy(deep=True)
    vectorizer = TfidfVectorizer(stop_words = 'english',decode_error='ignore',max_features=max_feature[index])
    vectorizer.fit(df2[column_name])
    vectorizers.append(vectorizer)

def text2features(df,text_column_names,vectorizers):
    df_vec_list = []
    for index, text_column_name in enumerate(text_column_names):
        vectorizer = vectorizers[index]
        print(index)
        X = vectorizer.transform(df[column_name])

        df_vec = pd.DataFrame(df['label'][:df.shape[0]])
        featureNameList = vectorizer.get_feature_names()
        for i in range(X.shape[1]):
            df_vec[featureNameList[i]] = X[:,i].toarray()
        print(df_vec) 
        df_vec_list.append(df_vec)
    df_concat = pd.concat([df_vec_list[0],df_vec_list[1].iloc[:,1:]],axis=1)
    return df_concat

# df_concat = text2features(df,text_column_names,vectorizers)


datasize:  27877
Drop empty content...
new datasize:  27871


In [34]:
%%time
#parameters
directory ="/Users/weichen/Downloads/"
file_name = "title_content_2gram_processed.csv"
text_column_names = ["title_new","content_new"]
label_column_name = "label"

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

def drop_nan_epmtystr(df,subset=['content_new']):
    df.replace(r'^\s*$', np.nan, regex=True,inplace=True)
    df.replace('', np.nan, inplace=True)
    df.dropna(subset=subset, inplace=True)
    return df

#read data
df = pd.read_csv(directory+file_name)
#always drop so code doesnt crash later
df = drop_nan_epmtystr(df,text_column_names)
df.head()

max_feature = [441,32]
# min_df = [.1,.001]
vectorizers = []
for index, column_name in enumerate(text_column_names):
    df2 = df.copy(deep=True)
    vectorizer = TfidfVectorizer(stop_words = 'english',decode_error='ignore',max_features=max_feature[index])
    vectorizer.fit(df2[column_name])
    vectorizers.append(vectorizer)

def text2features(df,text_column_names,vectorizers):
    df_vec_list = []
    for index, text_column_name in enumerate(text_column_names):
        vectorizer = vectorizers[index]
#         print(index)
        X = vectorizer.transform(df[column_name])

        df_vec = pd.DataFrame(df['label'][:df.shape[0]])
        featureNameList = vectorizer.get_feature_names()
        for i in range(X.shape[1]):
            df_vec[featureNameList[i]] = X[:,i].toarray()
#         print(df_vec) 
        df_vec_list.append(df_vec)
    df_concat = pd.concat([df_vec_list[0],df_vec_list[1].iloc[:,1:]],axis=1)
    return df_concat

df_concat = text2features(df,text_column_names,vectorizers)


CPU times: user 41.3 s, sys: 1.76 s, total: 43 s
Wall time: 45.3 s


In [40]:
print(df_concat.shape)
df_concat.head()


(27871, 474)


,label,access,accuse,act,administration,admit,age,ahead,air,al,...,think,time,trump,vote,want,way,woman,work,world,year
0,fake,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.325802,0.0,0.00000,0.000000,0.261244
1,fake,0.0,0.0,0.000000,0.237258,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.460895,0.344942
2,fake,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.144624,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000
3,fake,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.565723,0.000000,0.590405,0.000000,0.0,0.00000,0.000000,0.000000
4,fake,0.0,0.0,0.190163,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.208835,0.323498,0.000000,0.0,0.30898,0.000000,0.000000


In [36]:
print("writing to: ",to_file_name)
df_concat.to_csv(to_file_name, encoding='utf-8', index=False)

writing to:  /Users/weichen/Downloads/vector_title_content.csv


In [37]:
df_hot = pd.get_dummies(df[one_hot_column_name])
print(df_hot.shape)
df_concat_hot = pd.concat([df_concat,df_hot],axis=1)


(27871, 250)


In [ ]:
pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)[source]

In [ ]:
to_file_name = directory+ "vector_title_content_tfidf_publication_1hot.csv"#####
print("writing to: ",to_file_name)
df_concat_hot.to_csv(to_file_name, encoding='utf-8', index=False)

In [ ]:
for index, text_column_name in enumerate(text_column_names):
    print(df2[text_column_name].head())